In [ ]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/keras-idiomatic-programmer/blob/master/workshops/Idiomatic%20Programmer%20-%20handbook%203%20-%20Codelab%203.ipynb">
<img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>

# Idiomatic Programmer Code Labs

## Code Labs #3 - Get Familiar with Transfer Learning

## Prerequistes:

    1. Familiar with Python
    2. Completed Handbook 3/Part 12: Prebuilt Models & Transfer Learning

## Objectives:

    1. Use a prebuilt model and fully retrain.
    2. Use a prebuilt model and do transfer learning.

## Retraining with a Prebuilt Model

We will use a stock (prebuilt) ResNet50 model from the Keras model zoo repository, and:

    1. Replace the input layer to match the CIFAR-10 dataset.
    2. Replace the output layer to match the CIFAR-10 dataset.
    3. Leave weights to the default initialization (untrained).
    4. Train the model on CIFAR-10

You fill in the blanks (replace the ??), make sure it passes the Python interpreter.

In [ ]:
from keras import Model
from keras.layers import Dense

# We will use a stock ResNet50 model
from keras.applications import ResNet50

# Let's get a version of the ResNet50, which is missing the classifier (we will replace it), 
# has default initialization and input size will be for the CIFAR-10 dataset
# HINT: the top is the last layer (classifier) in the model, we don't want to include it.
model = ResNet50(include_top=??, input_shape=(32, 32, 3), pooling='avg')

# Add a classifier for 10 classes
# HINT: the input to this layer is the output from the above model.
outputs = Dense(10, activation='softmax')(??)
model = Model(model.input, outputs)

# Compile the model for training
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
from keras.utils import to_categorical
from keras.datasets import cifar10
import numpy as np

# Let's use the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel data
x_train = (x_train / 255.0).astype(np.float32)
x_test  = (x_test  / 255.0).astype(np.float32)

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

# ResNet50 will take along time to wrong an epoch (vs. our simple ConvNet)
# Will will use just a small amount for brevity (2%)
x_tmp = x_train[0:1000]
y_tmp = y_train[0:1000]

# Let's now train the prebuilt ResNet50 with the CIFAR-10 data
# HINT: You've been using this method a lot.
model.??(x_tmp, y_tmp, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

## Transfer Learning with a Prebuilt Model

Okay, our accuracy for the above is poor. Well, we only used 2% of the training data, and ran just a few epochs.

This time, let's go ahead and try transfer learning:

    1. Repeat getting and modifying the prebuilt ResNet50 as before.
    2. This time though initialize it with the learned weights from imagenet.
    3. Freeze the already trained ResNet50 layers.
    4. Train the top (classifier) layer we added.


In [ ]:

# Let's get a version of the ResNet50, which is missing the classifier (we will replace it), 
# has default initialization and input size will be for the CIFAR-10 dataset
# HINT: the parameter name refers to the weights
model = ResNet50(include_top=False, input_shape=(32, 32, 3), pooling='avg', ??='imagenet')

# Freeze the weights of the remaining layer
# HINT: the name of property (field) is something to do with being trainable.
for layer in model.layers:
    layer.?? = False
    

# Add a classifier for 10 classes
outputs = Dense(10, activation='softmax')(model.output)
model = Model(model.input, outputs)

# Compile the model for training
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
# Let's now train the prebuilt ResNet50 with the CIFAR-10 data
# This will train a lot faster since we are only updated the classifier layer -- so let's use all the data.
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

## End of Code Lab